In [84]:
from openml import tasks, study
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.preprocessing import LabelEncoder

/Users/tomashev-aa/dev-projects/autogluon/ag-venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [1]:
from pandas import read_csv, pivot_table

In [93]:
task = tasks.get_task(7593)

In [34]:
leaderboard = read_csv('examples/tabular/distill/leaderboard.csv')

In [35]:
leaderboard

,task,dataset,fold,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order,datetime
0,7592,adult,0,WeightedEnsemble_L2_DSTL_BASE,0.876561,0.876000,0.168071,0.071690,25.115468,0.004623,0.000423,0.207418,2,True,35,NaN
1,7592,adult,0,XGBoost_DSTL_BASE,0.875947,0.875600,0.104262,0.031340,3.793513,0.104262,0.031340,3.793513,1,True,33,NaN
2,7592,adult,0,CatBoost_DSTL_BASE,0.875537,0.873600,0.024060,0.015094,20.216331,0.024060,0.015094,20.216331,1,True,31,NaN
3,7592,adult,0,XGBoost_BAG_L1,0.875333,0.875788,0.955885,0.511863,31.660194,0.955885,0.511863,31.660194,1,True,10,NaN
4,7592,adult,0,WeightedEnsemble_L2,0.875333,0.875833,1.091788,0.712134,106.292356,0.003149,0.060658,12.344829,2,True,12,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3433,168912,sylvine,9,NeuralNetMXNet_BAG_L1,0.914062,0.933651,0.324430,0.231280,68.462225,0.324430,0.231280,68.462225,1,True,11,2021-03-19 03:47:59.711628
3434,168912,sylvine,9,KNeighborsDist_BAG_L1,0.775391,0.764744,1.387330,1.334928,0.020074,1.387330,1.334928,0.020074,1,True,6,2021-03-19 03:47:59.711628
3435,168912,sylvine,9,KNeighborsDist_BAG_L2,0.773438,0.759757,13.295004,9.733910,264.449185,1.461713,1.334364,0.025814,2,True,20,2021-03-19 03:47:59.711628
3436,168912,sylvine,9,KNeighborsUnif_BAG_L1,0.769531,0.763443,1.462471,1.345485,0.021480,1.462471,1.345485,0.021480,1,True,5,2021-03-19 03:47:59.711628


In [50]:
pivoted_lb = leaderboard[leaderboard.model.str.contains("WeightedEnsemble_L|BASE|KNOW")]\
    .pivot(index=['dataset', 'fold'], columns='model', values='score_test')
pivoted_lb['teacher_score'] = pivoted_lb['WeightedEnsemble_L3'].fillna(pivoted_lb['WeightedEnsemble_L2'])

In [53]:
pivoted_lb[pivoted_lb['WeightedEnsemble_L3'].isna()]

0.811594   
                                 4               0.884058            0.898551   
                                 5               0.927536            0.913043   
                                 6               0.855072            0.826087   
                                 7               0.913043            0.898551   
                                 8               0.898551            0.884058   
                                 9               0.855072            0.840580   
adult                            7               0.878788            0.874693   
blood-transfusion-service-center 0               0.773333            0.760000   
                                 1               0.760000            0.840000   
                                 2               0.733333            0.746667   
                                 3               0.813333            0.800000   
                                 4               0.813333            0.813333   
                                 5               0.800000            0.800000   
                                 6               0.853333            0.786667   
                                 7               0.786667            0.813333   
                                 8               0.797297            0.756757   
                                 9               0.743243            0.783784   
phoneme                          3               0.881701            0.890943   
                                 6               0.909259            0.927778   
                                 9               0.905556            0.907407   

model                                  LightGBM_DSTL_BASE  LightGBM_DSTL_KNOW  \
dataset                          fold                                           
Australian                       0               0.898551            0.855072   
                                 1               0.869565            0.840580   
                                 2               0.869565            0.869565   
                                 3               0.811594            0.840580   
                                 4               0.884058            0.884058   
                                 5               0.884058            0.869565   
                                 6               0.811594            0.797101   
                                 7               0.898551            0.884058   
                                 8               0.884058            0.869565   
                                 9               0.855072            0.840580   
adult                            7               0.874079            0.876945   
blood-transfusion-service-center 0               0.773333            0.773333   
                                 1               0.813333            0.840000   
                                 2               0.720000            0.746667   
                                 3               0.773333            0.800000   
                                 4               0.773333            0.786667   
                                 5               0.800000            0.786667   
                                 6               0.746667            0.786667   
                                 7               0.826667            0.826667   
                                 8               0.756757            0.756757   
                                 9               0.743243            0.783784   
phoneme                          3               0.885397            0.879852   
                                 6               0.924074            0.927778   
                                 9               0.905556            0.914815   

model                                  LinearModel_DSTL_BASE  \
dataset                          fold                          
Australian                       0                  0.855072   
                                 1                  0.898551   
    

In [65]:
pivoted_lb.assign(diff_cb=lambda x: x.teacher_score - x.CatBoost_DSTL_BASE,
                  sem_cb=lambda x: x.CatBoost_DSTL_BASE,
                  sem_teacher=lambda x: x.teacher_score)\
    .groupby(['dataset'])\
    .agg({'teacher_score': 'mean',
          'CatBoost_DSTL_KNOW': 'mean', 'CatBoost_DSTL_BASE': 'mean',
          'XGBoost_DSTL_KNOW': 'mean', 'XGBoost_DSTL_BASE': 'mean',
          'RandomForestMSE_DSTL_KNOW': 'mean', 'RandomForestMSE_DSTL_BASE': 'mean',
          'LinearModel_DSTL_KNOW': 'mean', 'LinearModel_DSTL_BASE': 'mean',
          'LightGBM_DSTL_KNOW': 'mean', 'LightGBM_DSTL_BASE': 'mean',
          'diff_cb': 'mean',
          'sem_cb': 'sem', 'sem_teacher': 'sem'})\
    .assign(know_score=lambda x: x.filter(like='KNOW').max(axis='columns'),
            base_score=lambda x: x.filter(like='BASE').max(axis='columns'),
            diff_know=lambda x: x.know_score - x.base_score,
            dif_base=lambda x: x.teacher_score - x.base_score).mean()

teacher_score                0.841943
CatBoost_DSTL_KNOW           0.840427
CatBoost_DSTL_BASE           0.839528
XGBoost_DSTL_KNOW            0.823578
XGBoost_DSTL_BASE            0.817635
RandomForestMSE_DSTL_KNOW    0.813610
RandomForestMSE_DSTL_BASE    0.817435
LinearModel_DSTL_KNOW        0.786609
LinearModel_DSTL_BASE        0.777444
LightGBM_DSTL_KNOW           0.838465
LightGBM_DSTL_BASE           0.836852
diff_cb                      0.002415
sem_cb                       0.004085
sem_teacher                  0.004200
know_score                   0.841208
base_score                   0.840960
diff_know                    0.000248
dif_base                     0.000983
dtype: float64

In [104]:
lb_reg = read_csv('examples/tabular/distill/leaderboard_regression.csv')

In [105]:
pivoted_lb_reg = lb_reg[lb_reg.model.str.contains("WeightedEnsemble_L|BASE|KNOW")]\
    .pivot(index=['dataset', 'fold'], columns='model', values='score_test')
pivoted_lb_reg['teacher_score'] = pivoted_lb_reg['WeightedEnsemble_L3'].fillna(pivoted_lb_reg['WeightedEnsemble_L2'])

In [106]:
pivoted_lb_reg.assign(sem_teacher=lambda x: x.teacher_score)\
    .groupby(['dataset'])\
    .agg({'teacher_score': 'mean', 'sem_teacher': 'sem',
          'CatBoost_DSTL_KNOW': 'mean', 'CatBoost_DSTL_BASE': 'mean',
          'XGBoost_DSTL_KNOW': 'mean', 'XGBoost_DSTL_BASE': 'mean',
          'RandomForest_DSTL_KNOW': 'mean', 'RandomForest_DSTL_BASE': 'mean',
          'LinearModel_DSTL_KNOW': 'mean', 'LinearModel_DSTL_BASE': 'mean',
          'LightGBM_DSTL_KNOW': 'mean', 'LightGBM_DSTL_BASE': 'mean'})\
    .assign(know_score=lambda x: x.filter(like='KNOW').max(axis='columns'),
            base_score=lambda x: x.filter(like='BASE').max(axis='columns'),
            diff_know=lambda x: x.know_score - x.base_score,
            dif_base=lambda x: x.teacher_score - x.base_score)#.mean()

,teacher_score,sem_teacher,CatBoost_DSTL_KNOW,CatBoost_DSTL_BASE,XGBoost_DSTL_KNOW,XGBoost_DSTL_BASE,RandomForest_DSTL_KNOW,RandomForest_DSTL_BASE,LinearModel_DSTL_KNOW,LinearModel_DSTL_BASE,LightGBM_DSTL_KNOW,LightGBM_DSTL_BASE,know_score,base_score,diff_know,dif_base
dataset,,,,,,,,,,,,,,,,
bostonHousing,0.886165,0.021189,0.867358,0.873995,0.861543,0.857088,0.839464,0.853988,0.725910,0.727173,0.858888,0.864340,0.867358,0.873995,-0.006637,0.012170
concrete,0.945851,0.006535,0.939791,0.939659,0.925257,0.923106,0.898139,0.900628,0.752462,0.752689,0.930436,0.927046,0.939791,0.939659,0.000132,0.006193
energy,0.999158,0.000162,0.998832,0.998783,0.998713,0.998504,0.997122,0.997108,0.912076,0.912091,0.997675,0.997651,0.998832,0.998783,0.000048,0.000375
kin8nm,0.925179,0.001648,0.882472,0.877199,0.808465,0.796376,0.727806,0.715637,0.411497,0.411536,0.834864,0.826109,0.882472,0.877199,0.005273,0.047980
naval-propulsion-plant,0.999640,0.000028,0.998517,0.998486,0.995699,0.995555,0.998149,0.998004,0.802867,0.802666,0.077901,0.077832,0.998517,0.998486,0.000031,0.001154
power-plant,0.972497,0.001678,0.970775,0.969174,0.971111,0.969735,0.962428,0.961935,0.928447,0.928437,0.968709,0.967585,0.971111,0.969735,0.001376,0.002762
protein-tertiary-structure,0.721706,0.003597,0.658986,0.652761,0.646041,0.641878,0.674580,0.676375,0.279816,0.280005,0.645431,0.643827,0.674580,0.676375,-0.001795,0.045331
wine-quality-red,0.504223,0.012622,0.461043,0.443744,0.440953,0.415284,0.455565,0.466643,0.358316,0.357081,0.437987,0.428413,0.461043,0.466643,-0.005601,0.037579
yacht,0.997213,0.000680,0.982895,0.984501,0.996526,0.996948,0.993149,0.993548,0.615842,0.615746,0.933497,0.933476,0.996526,0.996948,-0.000421,0.000265
